In [35]:
import os
import pandas as pd
import joblib
from tqdm.notebook import tqdm
from lightgbm import LGBMClassifier

In [36]:
out_dir = "output/classifiers/"
os.makedirs(out_dir, exist_ok = True)

In [2]:
df = pd.read_parquet("output/features/features_nsrr_shhs2.parquet")

In [3]:
df_demo = pd.read_csv("output/demographics/shhs_demo.csv")
# Remove columns that are already present in `df`
df_demo.drop(columns=['male', 'age'], inplace=True)
df_demo

,subj,bmi,ahi,ethnicity,set,hypertension
0,200077,23.388687,9.738220,caucasian,training,0.0
1,200078,30.211833,19.685039,caucasian,training,1.0
2,200079,35.451050,26.000000,other,testing,0.0


#### Build Training Features

In [18]:
cols_all = df.columns
cols_time = cols_all[cols_all.str.startswith('time_')].tolist()
# EEG also includes the time columns
cols_eeg = cols_all[cols_all.str.startswith('eeg_')].tolist() + cols_time  
cols_eog = cols_all[cols_all.str.startswith('eog_')].tolist()
cols_emg = cols_all[cols_all.str.startswith('emg_')].tolist()
cols_demo = ['age', 'male']

In [20]:
# Define predictors
X_all = {
    'eeg': df[cols_eeg],
    'eeg+demo': df[cols_eeg + cols_demo],
    
    'eeg+eog': df[cols_eeg + cols_eog],
    'eeg+eog+demo': df[cols_eeg + cols_eog + cols_demo],
    
    'eeg+emg': df[cols_eeg + cols_emg],
    'eeg+emg+demo': df[cols_eeg + cols_emg + cols_demo],
    
    'eeg+eog+emg': df[cols_eeg + cols_eog + cols_emg],
    'eeg+eog+emg+demo': df[cols_eeg + cols_eog + cols_emg + cols_demo],
}

# Define target and groups
y = df['stage']
subjects = df.index.get_level_values(0).to_numpy()

In [21]:
# Export a full list of features
features = pd.Series(X_all['eeg+eog+emg+demo'].columns, name="Features")
features.to_csv("features.csv", index=False)

In [26]:
# Define hyper-parameters
params = dict(
    boosting_type='gbdt',
    n_estimators=400,
    max_depth=5,
    num_leaves=90,
    colsample_bytree=0.5,
    importance_type='gain',
    class_weight=None
)

In [37]:
# Parallel processing when building the trees.
params['n_jobs'] = 8

# Loop across combs of predictors
for name, X in tqdm(X_all.items()):
    
    # Fit
    clf = LGBMClassifier(**params)
    clf.fit(X, y)

    # Print the accuracy on the training dataset: shouldn't be too high..!
    print("%s (%i features) - training accuracy: %.3f" % 
        (name, X.shape[1], clf.score(X, y)))
    
    # Export trained classifier
    if params['class_weight'] is not None:
        fname = out_dir + 'clf_%s_lgb_%s_%s.joblib' % \
        (name, params['boosting_type'], class_weight)
    else:
        fname = out_dir + 'clf_%s_lgb_%s.joblib' % \
        (name, params['boosting_type'])
        
    # Export model
    joblib.dump(clf, fname, compress=True)
    
    # Also save directly to YASA
    # out_dir_yasa = "/Users/raphael/GitHub/yasa/yasa/classifiers/"
    # fname_yasa = out_dir_yasa + 'clf_%s_lgb.joblib' % name
    # joblib.dump(clf, fname_yasa, compress=True)
    
    # Features importance (full model only)
    if name == "eeg+eog+emg+demo":
        # Export LGBM feature importance
        df_imp = pd.Series(clf.feature_importances_, index=clf.feature_name_, name='Importance').round()
        df_imp.sort_values(ascending=False, inplace=True)
        df_imp.index.name = 'Features'
        df_imp.to_csv(fname[:-7] + ".csv")

eeg (65 features) - training accuracy: 1.000
eeg+demo (67 features) - training accuracy: 1.000
eeg+eog (116 features) - training accuracy: 1.000
eeg+eog+demo (118 features) - training accuracy: 1.000
eeg+emg (98 features) - training accuracy: 1.000
eeg+emg+demo (100 features) - training accuracy: 1.000
eeg+eog+emg (149 features) - training accuracy: 1.000
eeg+eog+emg+demo (151 features) - training accuracy: 1.000

